In [3]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

from pathlib import Path
import scipy
import numpy as np
import os,sys

# for windows
'''
currentdir = Path.cwd()
sys.path.insert(0,str(currentdir)+'\\utils') 
'''

# for ubuntu
os.chdir('./utils')
import skseq
import skseq.sequences.structured_perceptron as spc
from utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


This notebook loads the data, loads the fitted models from disk and evaluates the models.

## Loading Data

In [4]:
corpus = NerCorpus()

In [6]:
data_path = "../data"

#data_path = parentdir + data_path

train_seq = corpus.read_sequence_list(data_path + "/train_data_ner.csv", 
                                            max_sent_len=100)

test_seq = corpus.read_sequence_list(data_path + "/test_data_ner.csv",
                                           max_sent_len=100)

tiny_test_seq = corpus.read_sequence_list(data_path + "/tiny_test.csv", 
                                          max_sent_len=100)

In [7]:
tiny_test_metrics = {'Accuracy':[],'F1':[]}
names = []

## Structure Perceptron

### Structure Perceptron w/ given features

In [9]:
feature_mapper = skseq.sequences.id_feature.IDFeatures(train_seq)
feature_mapper.build_features()

sp = spc.StructuredPerceptron(corpus.word_dict, corpus.tag_dict, feature_mapper)
sp.load_model(dir="../fitted_models/perceptron_5_iter_given")

In [10]:
#pred_train = sp.viterbi_decode_corpus(train_seq)
#pred_test  = sp.viterbi_decode_corpus(test_seq)
pred_tiny_test  = sp.viterbi_decode_corpus(tiny_test_seq)

In [11]:
acc, f1 = Evaluate_metrics(tiny_test_seq.seq_list, pred_tiny_test, corpus)
tiny_test_metrics['Accuracy'].append(acc)
tiny_test_metrics['F1'].append(f1)
names.append('SP-given')

### Structure Perceptron w/ extra features

In [ ]:
from skseq.sequences.extended_features import *

feature_mapper_ext = Extended_Features(train_seq)
feature_mapper_ext.build_features()

sp = spc.StructuredPerceptron(corpus.word_dict, corpus.tag_dict, feature_mapper_ext)
sp.load_model(dir="./fitted_models/perceptron_5_iter_extra")

In [ ]:
pred_tiny_test  = sp.viterbi_decode_corpus(tiny_test_seq)

In [ ]:
acc, f1 = Evaluate_metrics(tiny_test_seq.seq_list, pred_tiny_test, corpus)
tiny_test_metrics['Accuracy'].append(acc)
tiny_test_metrics['F1'].append(f1)
names.append('SP-extra')

## Deep Learning Approach